# HERA RFI Analysis



# Load Files

In [1]:
%matplotlib inline
from pyuvdata import UVData
import numpy as np
import itertools
import os

# load data here



#items = os.listdir(".")
items=os.listdir('/lustre/aoc/projects/hera/ajosaiti/May7_RFI/2458610/')


uvfiles = []
for names in items:
    if names.endswith(".HH.uvh5"):
        uvfiles.append(names)
print(str(uvfiles))

uvd = UVData()
uvd.read_uvh5(uvfiles, axis='blt')

ImportError: No module named pyuvdata

In [2]:
# your answers here (hint: look at the attributes of your uvd object)

print 'There are', len(uvd.antenna_numbers), 'antennas and they are:', uvd.antenna_numbers

print 'There are', uvd.Nbls, 'baselines, which corresponds to N(N-1)/2 cross-correlations plus N auto-correlations.'

print 'There are', uvd.Nfreqs, 'frequencies, covering a range of', \
    np.min(uvd.freq_array)/1e6, 'MHz to',np.max(uvd.freq_array)/1e6,'MHz.'

print 'The width of each frequency channel is', (np.max(uvd.freq_array)-np.min(uvd.freq_array))/uvd.Nfreqs/1e6, 'MHz'

print 'There are', uvd.Ntimes, 'time integrations.'

print 'The LSTs are', np.unique(uvd.lst_array)*12/np.pi, 'in hours.'

There are

NameError: name 'uvd' is not defined

# B) Visualizing Data and Flags

In [3]:
import matplotlib.pyplot as plt
#%matplotlib notebook

antpos, ants = uvd.get_ENU_antpos() # this returns coordinates of each antenna and a list of the antennas

plt.figure()
plt.scatter(antpos[:,0], antpos[:,1], marker='.', color='k', s=3000) # plot the antenna positions with black circles
for aa,ant in enumerate(ants): # loop over antennas
    plt.text(antpos[aa,0], antpos[aa,1], ants[aa], color='w', va='center', ha='center') # label antenna numbers
plt.xlabel('X-position (m)')
plt.ylabel('Y-position (m)')
plt.axis('equal');

NameError: name 'uvd' is not defined

Let's now access the visibility data in our file for baseline (65,71), a long E/W baseline, and plot the real and imaginary part of the visibility for the first time integration.

Exercise: Choose a few frequency channels and plot them as a function of time.

# C) Waterfall Plots

In [4]:
import matplotlib 

ants_select = np.array([23,25,26])
ant_combination = np.array( list( itertools.combinations(ants_select, 2) ) )
for ant in ants_select:
    ant_combination= np.append(ant_combination, [[ant,ant]], axis=0)
    
np.shape(ant_combination)

ii=0
while ii < np.size(ant_combination[:,0]):
    pols = np.array(['xx','yy'])
    for pol in pols:
        key=(ant_combination[ii,0],ant_combination[ii,1],str(pol))
        vis=uvd.get_data(key)
        plt.figure(figsize=(20,8))
        
        plt.subplot(121) #For amplitude
        plt.imshow(np.abs(vis), aspect='auto',extent=(np.min(uvd.freq_array[0])/1e6,np.max(uvd.freq_array[0])/1e6,np.max(np.unique(uvd.lst_array))*12/np.pi,np.min(np.unique(uvd.lst_array))*12/np.pi))
        plt.colorbar(label='Visibility Amplitude (Jy)')
        plt.xlabel('Frequency (MHz)')
        plt.ylabel('LST (hours)')
        plt.title('Ant ('+str(ant_combination[ii,0])+', '+str(ant_combination[ii,1])+') Pol: '+str(pol)+', Amplitude')

        plt.tight_layout();
    ii = ii+1


NameError: name 'np' is not defined

# Median and Mean Statistics, XRFI and Boxplot 

In [5]:
import xrfi_funcs
key=(26,26,'xx')
vis=uvd.get_data(key)
powers_per_time = np.abs(vis)
#calc XRFI-flagged visibility
XRFI_mask = xrfi_funcs.xrfi(powers_per_time)
powers_per_time_XRFI=np.ma.array(powers_per_time,mask=XRFI_mask)
total_power_chan_XRFI=10.0 * np.log10(np.sum(np.power(10.0, np.ma.masked_array(powers_per_time_XRFI).filled(np.ma.masked_array(powers_per_time_XRFI).mean()) / 10.0),0))

'''
x_boxplot=list(uvd.freq_array[0]/1.e6)
meanlineprops = dict(linestyle='--', linewidth=2.5, color='green')
medianprops = dict(linestyle='-.', linewidth=2.5, color='firebrick')
#plt.figure(1,figsize=(25,25))
#plt.boxplot(abs(vis),widths=0.05,meanprops=meanlineprops,showfliers=False,positions=x_boxplot,meanline=True,whis='range',patch_artist=True,manage_xticks=False)

#plt.title('Boxplot (Pre XRFI): Median (Red), Mean (Green), 25/75th Percentiles, and Max/Min.')
#plt.xticks(rotation='vertical')
#plt.xticks(np.linspace(np.min(uvd.freq_array[0])/1e6,np.max(uvd.freq_array[0])/1e6,50),rotation='vertical')
#ylim_min = 1.05*np.min(np.min(powers_per_time,axis=0))#ATJ
#ylim_max = 0.95*np.max(np.max(powers_per_time,axis=0)) #ATJ
#plt.ylim([ylim_min,ylim_max])#ATJ

plt.figure(1,figsize=(25,25))
pwr_xrfi_vec_seq = [[y for y in row if y] for row in powers_per_time_XRFI.T]
plt.figure(1,figsize=(25,25))
plt.boxplot(pwr_xrfi_vec_seq,widths=0.05,meanprops=meanlineprops,showfliers=False,positions=x_boxplot,meanline=True,whis='range',patch_artist=True,manage_xticks=False)
plt.title('Boxplot (Post XRFI): Median (Red), Mean (Green), 25/75th Percentiles, and Max/Min.')
plt.xticks(np.linspace(np.min(uvd.freq_array[0])/1e6,np.max(uvd.freq_array[0])/1e6,50),rotation='vertical')
#plt.ylim([ylim_min, ylim_max]) 

K = 8 #the box size for taking medians in xrfi, see xrfi_funcs.xrfi()
arr2D_pow = np.tile( np.max( np.ma.masked_array(powers_per_time_XRFI).filled(np.ma.masked_array(powers_per_time_XRFI).mean() ),axis=0) , (int(2.*K +2),1))
print('For XRFI algorithm: K = '+str(K)+', shape of xrfi input (arr2D_pow): '+str(np.shape(arr2D_pow)))
xrfi_mask = xrfi_funcs.xrfi( arr2D_pow, K=K )
masked_y = np.ma.masked_array( arr2D_pow[0,:][np.where(xrfi_mask[0,:])])
print('After running XRFI on the full data set, if I run a 1D XRFI on the max vals per channel, I get '+str(np.size(np.where(xrfi_mask[0,:])))+' channels that still could be RFI'  )

'''

NameError: name 'uvd' is not defined

# Neighboring channel analysis - Antenna 26, Pol XX

Post XRFI: detect low-level RFI. Try calculating ratio of median over all time (persistent RFI) and std over all time (for transient RFI) of neighboring channels. Which channels have the highest ratios? Also perform an integrated-overall-all-time, per channel comparison. 

In [6]:
figx=30;figy=30; 
xticks=np.linspace(np.min(uvd.freq_array[0])/1e6,np.max(np.array(uvd.freq_array[0]/1.e6)[1:]),50);



mean = np.mean(powers_per_time_XRFI,axis=0)
arr_max = np.max(powers_per_time_XRFI,axis=0)
arr_min = np.min(powers_per_time_XRFI,axis=0)

plt.figure(figsize=(figx,figy))
plt.plot( np.array(uvd.freq_array[0]/1.e6), mean,'k-',label='Mean')
plt.plot( np.array(uvd.freq_array[0]/1.e6), arr_max,'r-',label='Max')
plt.plot( np.array(uvd.freq_array[0]/1.e6), arr_min,'b-',label='Min')
plt.title('Mean, Min, and Max of post-XRFI data')
plt.legend()
plt.xticks(xticks,rotation='vertical')

ratio_mean = np.divide(mean[1:], mean[:-1])
plt.figure(figsize=(figx,figy))
plt.plot( np.array(uvd.freq_array[0]/1.e6)[1:], ratio_mean)
plt.title('Ratio of Means (Neighboring Channel)')
plt.xticks(xticks,rotation='vertical')


plt.figure(figsize=(figx,figy))
plt.plot( np.array(uvd.freq_array[0]/1.e6)[1:], np.log(ratio_mean-1.))
plt.title('log(Ratio of Means -1.) (Neighboring Channel)')
plt.xticks(xticks,rotation='vertical')

median = np.median(powers_per_time_XRFI,axis=0)
ratio_median = np.divide(median[1:], median[:-1])
plt.figure(figsize=(figx,figy))
plt.plot( np.array(uvd.freq_array[0]/1.e6)[1:], ratio_median)
plt.title('Ratio of Medians (Neighboring Channel)')
plt.xticks(xticks,rotation='vertical')

plt.figure(figsize=(figx,figy))
plt.plot( np.array(uvd.freq_array[0]/1.e6)[1:], np.log(ratio_median-1.))
plt.title('log(Ratio of Medians -1.) (Neighboring Channel)')
plt.xticks(xticks,rotation='vertical')


std = np.std(powers_per_time_XRFI,axis=0)
ratio_std = np.divide(std[1:], std[:-1])
plt.figure(figsize=(figx,figy))
plt.plot( np.array(uvd.freq_array[0]/1.e6)[1:], ratio_std)
plt.title('Ratio of Standard Deviations (Neighboring Channel)')
plt.xticks(xticks,rotation='vertical')

plt.figure(figsize=(figx,figy))
plt.plot( np.array(uvd.freq_array[0]/1.e6)[1:], np.log(ratio_std-1.))
plt.title('log(Ratio of Standard Deviations -1.) (Neighboring Channel)')
plt.xticks(xticks,rotation='vertical')


std = np.std(powers_per_time_XRFI,axis=0)
ratio_std = np.divide(std[1:], std[:-1])
plt.figure(figsize=(figx,figy))
plt.plot( uvd.freq_array[0]/1.e6, std)
plt.title('Standard Deviation Per Channel')
plt.xticks(xticks,rotation='vertical')


NameError: name 'np' is not defined